In [17]:
from tensorflow.keras.applications import VGG16, ResNet50
from tensorflow.keras import layers, models
import cv2 as cv
import zipfile
import os

In [4]:
!wget /content/Train_Warwick.zip
!wget /content/Test_Warwick.zip

# Extrai o conteúdo do arquivo zip
!unzip Train_Warwick.zip
!unzip Test_Warwick.zip

/content/Train_Warwick.zip: Scheme missing.
/content/Test_Warwick.zip: Scheme missing.
Archive:  Train_Warwick.zip
   creating: Train_4cls_amostra/
   creating: Train_4cls_amostra/0/
  inflating: Train_4cls_amostra/0/46_HER2_61709.png  
  inflating: Train_4cls_amostra/0/46_HER2_64186.png  
  inflating: Train_4cls_amostra/0/13_HER2_10243.png  
  inflating: Train_4cls_amostra/0/46_HER2_58961.png  
  inflating: Train_4cls_amostra/0/46_HER2_64180.png  
  inflating: Train_4cls_amostra/0/46_HER2_55795.png  
  inflating: Train_4cls_amostra/0/18_HER2_25188.png  
  inflating: Train_4cls_amostra/0/13_HER2_11145.png  
  inflating: Train_4cls_amostra/0/13_HER2_2910.png  
  inflating: Train_4cls_amostra/0/01_HER2_4962.png  
  inflating: Train_4cls_amostra/0/18_HER2_30785.png  
  inflating: Train_4cls_amostra/0/13_HER2_11144.png  
  inflating: Train_4cls_amostra/0/01_HER2_4153.png  
  inflating: Train_4cls_amostra/0/57_HER2_6110.png  
  inflating: Train_4cls_amostra/0/01_HER2_7841.png  
  inflating:

In [14]:
def load_images_from_folder(folder):
  '''Carrega as imagens dos diretorios em uma lista com o nome do arquivo e classe'''
  images = []
  for filename in os.listdir(folder):
    img = cv.imread(os.path.join(folder,filename))
    if img is not None:
      images.append((img, filename, i))
  return images

def resize_images(images):
    '''Redimensiona as imagens para 224x224 como esperado por VGG16, ResNet50'''
    resized_images = []
    for image, filename, i in images:
      resized_image = cv.resize(image, (224, 224))
      resized_images.append((resized_image, filename, i))
    return resized_images

In [21]:
# carregando as imagens em 2 listas
temp_train_images = []
test_images = []

for i in range(4):
  temp_train_images += load_images_from_folder('Train_4cls_amostra/' + str(i) + '/')

for i in range(4):
  test_images += load_images_from_folder('Test_4cl_amostra/' + str(i) + '/')

  ''' separando em treino de validação, separando as imagens de 1 paciente por
categoria (aprox. 30 imagens por paciente) temos ~20% da base de treino
Pacientes escolhidos: 01, 14, 04 e 06'''

train_images = []
val_images = []

for image, filename, cat in temp_train_images:
  if filename.startswith('01'):
    val_images.append((image, filename, cat))
  elif filename.startswith('14'):
    val_images.append((image, filename, cat))
  elif filename.startswith('04'):
    val_images.append((image, filename, cat))
  elif filename.startswith('06'):
    val_images.append((image, filename, cat))
  else:
    train_images.append((image, filename, cat))

print(f'Imagens de treino: {len(train_images)}')
print(f'Imagens de validação: {len(val_images)}')
print(f'Imagens de teste: {len(test_images)}')

Imagens de treino: 477
Imagens de validação: 116
Imagens de teste: 371


In [22]:
#redimencionando as imagens

train_images = resize_images(train_images)
val_images = resize_images(val_images)
test_images = resize_images(test_images)

In [23]:
# carregando as redes com transfer learning sem as camadas fuly connected

base_model_vgg16 = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model_resnet50 = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

In [18]:
# Adicionar camadas fully connected no VGG16
model_vgg16 = models.Sequential([
    base_model_vgg16,
    layers.Flatten(),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(4, activation='softmax')  # 4 classes
])